In [48]:
import flighty as f

f.create_endpoint(name='doc-rec')
f.create_artifact(path='./README.md', name='xgboost', version=0)
f.create_handler(endpoint='doc-rec', name='xgboost', 
  code_path='./code_sample', artifacts=[{'name': 'xgboost', 'version': 0}])

{'name': 'doc-rec', 'url': 'http://127.0.0.1/doc-rec'}
{'name': 'xgboost',
 'path': '/code/flighty-files/xgboost/0',
 'type': 'model',
 'version': 0}
{'name': 'doc-rec-xgboost-0-code',
 'path': '/code/flighty-files/doc-rec-xgboost-0-code/0',
 'type': 'code',
 'version': 0}
{'docker_image': 'docker.io/gvashishtha/flighty:model_server',
 'endpoint': 'doc-rec',
 'name': 'xgboost',
 'version': 0}


In [50]:
response = f.invoke(endpoint='doc-rec', body={'data': {'id': 1, 'user': "john"}})
response.text
response.status_code

200

In [51]:
f.list_handlers()

[{'docker_image': 'docker.io/gvashishtha/flighty:model_server',
  'endpoint_id': 8,
  'id': 12,
  'name': 'xgboost',
  'prod_traffic': 100,
  'shadow_traffic': 0,
  'version': 0}]


In [52]:
f.delete_handler(endpoint='doc-rec', name='xgboost', version=0)
f.delete_endpoint(name='doc-rec')
f.delete_artifact(name='xgboost', version=0)

# delete_artifact(name='code-artifact', version=1)
# delete_artifact(name='model-artifact', version=2)
# delete_artifact(name='xgboost_code', version=0)

None
'Artifact doc-rec-xgboost-0-code with version 0 successfully deleted.'
'Endpoint doc-rec successfully deleted.'
'Artifact xgboost with version 0 successfully deleted.'


In [54]:
f.list_artifacts()
f.list_endpoints()
f.list_handlers()

[]
[]
[]
